In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [2]:
class Genetic():
    """
    Parameters:
    -----------
    population_size: int
    mutation_prob: float
    ...
    """
    def __init__(self, classifier, X_train, y_train, X_test, y_test, feature_costs, feature_names, 
                     population_size, crossover_prob, mutation_prob):
        self.classifier = classifier
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.feature_costs = feature_costs
        self.feature_names = feature_names
        self.num_features = X_train.shape[1]
        self.population_size = population_size
        self.crossover_prob = crossover_prob
        self.mutation_prob = mutation_prob

    def _initialize(self):
        """ 
        Initialize population 
        """
        self.population = []
        for _ in range(self.population_size):
            individual = "".join(np.random.choice(['0', '1'], size=self.num_features))
            self.population.append(individual)
    
    def _ones_indexes(self, individual):
        """
        Return a list of indexes of '1's in individual string
        """
        ones = [i for i in range(len(individual) - 1) if individual[i] == '1']
        if individual[20] == '1':
            if individual[18] == '0' or individual[19] == '0':
                ones.append(20)
        return ones
        
    def _fitness(self):
        """ 
        Calculate the fitness (cost) of each individual in the population 
        """
        alpha = 2 # coefficient of classification error in fitness function
        beta = 1 # coefficient of feature selection cost in fitness function
        total_sum_feature_costs = sum(self.feature_costs)
        population_fitness = [] # list of fitness values of all individuals
        for individual in self.population:
            ones_list = self._ones_indexes(individual) # which features are selected
            if len(ones_list) > 0:
                # limit X_train and X_test to the selected features
                X_train = self.X_train[:, ones_list]
                X_test = self.X_test[:, ones_list]
                self.classifier.fit(X=X_train, y=self.y_train) 
                classification_error = 1 - self.classifier.score(X=X_test, y=self.y_test)
                feature_selection_cost = sum(self.feature_costs[ones_list]) / total_sum_feature_costs
                cost = (alpha * classification_error + beta * feature_selection_cost) / (alpha + beta)
                fitness = cost
            else: # no feature selected => max cost
                fitness = 1
            population_fitness.append(fitness)
        return population_fitness
    
    def _mutate(self, individual):
        """ 
        Flip one bit of individual with probability self.mutation_prob
        """
        if np.random.random() < self.mutation_prob:
            mutation_point = np.random.randint(0, len(individual))
            individual = individual[:mutation_point] + \
                            str(1 - int(individual[mutation_point])) + individual[mutation_point+1:]
        return individual

    def _crossover(self, parent1, parent2):
        """ 
        Create children from parents by single-point crossover 
        """
        if np.random.random() < self.crossover_prob: # consider crossover probability
            # randomly select crossover point
            crossover_point = np.random.randint(0, len(parent1))
            child1 = parent1[:crossover_point] + parent2[crossover_point:]
            child2 = parent2[:crossover_point] + parent1[crossover_point:]
        else: # no crossover
            child1 = parent1
            child2 = parent2
        return child1, child2

    def run(self, max_iterations):
        
        self._initialize() # initialize the population
        
        last_fittest_individuals = [] # a list of fittest individuals, used in stopping criteria

        for iteration in range(max_iterations):
            population_fitness = self._fitness() # compute fitness of all individuals

            fittest_individual = self.population[np.argmin(population_fitness)]
            best_fitness = min(population_fitness)
            last_fittest_individuals.append(fittest_individual)
            
            # stopping criteria: 3 last fittest individual remain unchanged
            if last_fittest_individuals[-2:] == last_fittest_individuals[-3:-1]:
                break

            sum_population_fitness = sum(population_fitness) + 1
            parent_probs = np.array([sum_population_fitness - fit for fit in population_fitness])
            parent_probs = parent_probs / sum(parent_probs)

            new_population = []
            for i in np.arange(0, self.population_size, 2):
                parent1, parent2 = np.random.choice(self.population, size=2, p=parent_probs, replace=True)
                child1, child2 = self._crossover(parent1, parent2)
                child1 = self._mutate(child1)
                child2 = self._mutate(child2)
                new_population += [child1, child2]

            print ("Iteration: %d, Best Fitness: %.2f, Fittest Individual: %s]" % (iteration, best_fitness, fittest_individual))
            self.population = new_population

        self.report_results(iteration, fittest_individual)
        return
        
    def report_results(self, last_iteration, fittest_individual):
        """
        Print features selected by the genetic algorithm, the total cost of the selected features,
        and the training and test set accuracies.
        """
        print ("\nIteration: %d, Final Best Individual: '%s']" % (last_iteration, fittest_individual))
        ones_list = self._ones_indexes(fittest_individual) # which features are selected
        print('\nSelected Features:', feature_names[ones_list])
        feature_selection_cost = sum(self.feature_costs[ones_list]) / sum(self.feature_costs)
        print('\nTotal cost of the selected features:', feature_selection_cost)
        X_train = self.X_train[:, ones_list]
        X_test = self.X_test[:, ones_list]
        self.classifier.fit(X=X_train, y=self.y_train)
        test_score = self.classifier.score(X=X_test, y=self.y_test)
        print('\nTest accuracy:', test_score)
        return
        

In [3]:
def read_data():
    """
    Read train data, test data, and feature costs
    """
    train = pd.read_csv('ann-train.data', delimiter=' ', header=None)
    X_train = train.iloc[:, 0:21].values
    y_train = train[21]
    y_train = y_train.values.reshape(y_train.shape[0], 1)
    
    test = pd.read_csv('ann-test.data', delimiter=' ', header=None)
    X_test = test.iloc[:, 0:21].values
    y_test = test[21]
    y_test = y_test.values.reshape(y_test.shape[0], 1)
    
    features = pd.read_csv('ann-thyroid.cost', delimiter='\t+', header=None, engine='python')
    feature_names = features[0].values
    feature_names = np.array([name.split(':')[0] for name in feature_names])
    feature_costs = features[1].values
    feature_costs = np.append(feature_costs, feature_costs[-1] + feature_costs[-2])
    
    return X_train, y_train, X_test, y_test, feature_costs, feature_names

In [4]:
X_train, y_train, X_test, y_test, feature_costs, feature_names = read_data()

clf = DecisionTreeClassifier()

population_size = 500
crossover_prob = 0.8
mutation_prob = 0.05
max_iterations = 500

genetic = Genetic(clf, X_train, y_train, X_test, y_test, feature_costs, feature_names, 
                                            population_size, crossover_prob, mutation_prob)
genetic.run(max_iterations=max_iterations)

Iteration: 0, Best Fitness: 0.06, Fittest Individual: 011100000100001000000]
Iteration: 1, Best Fitness: 0.07, Fittest Individual: 010100100110100000000]
Iteration: 2, Best Fitness: 0.06, Fittest Individual: 000100001010110000000]
Iteration: 3, Best Fitness: 0.06, Fittest Individual: 011101000000001000000]
Iteration: 4, Best Fitness: 0.07, Fittest Individual: 010001100011101000000]
Iteration: 5, Best Fitness: 0.06, Fittest Individual: 000000001000010000000]
Iteration: 6, Best Fitness: 0.06, Fittest Individual: 000000001000010000000]
Iteration: 7, Best Fitness: 0.06, Fittest Individual: 001000100001110000000]
Iteration: 8, Best Fitness: 0.06, Fittest Individual: 100101000000001000000]
Iteration: 9, Best Fitness: 0.06, Fittest Individual: 100101000000001000000]
Iteration: 10, Best Fitness: 0.07, Fittest Individual: 000001010100010100000]
Iteration: 11, Best Fitness: 0.07, Fittest Individual: 000101011000011000000]
Iteration: 12, Best Fitness: 0.07, Fittest Individual: 0111100001001000000